# 🍎🥕 Transfer Learning para Clasificación de Frutas y Verduras

**Proyecto 3 - Inteligencia Artificial**  
**INFO1185 - Prof. Dr. Ricardo Soto Catalán**  
**Fecha: 28 de Noviembre de 2025**

## Objetivo
Aplicar Transfer Learning utilizando modelos preentrenados de PyTorch para clasificar frutas y verduras, comparando dos variantes:
- **Versión 1**: Clasificador simple (FC única)
- **Versión 2**: Clasificador extendido tipo embudo (con BN y Dropout)

## Dataset
Fruits and Vegetables Image Recognition Dataset de Kaggle

## 📚 1. Importaciones y Configuración

In [ ]:
# Instalación de dependencias (si es necesario)
!pip install torch torchvision matplotlib seaborn scikit-learn pillow kaggle

# Importaciones principales
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision
from torchvision import datasets, models, transforms

# Para visualización y análisis
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from PIL import Image
import os
from pathlib import Path
import copy
import time
from collections import defaultdict

# Para métricas
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score

# Configuración
import warnings
warnings.filterwarnings('ignore')

# Verificar GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Dispositivo utilizado: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memoria GPU disponible: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')

## 🔧 2. Configuración de Hiperparámetros

In [ ]:
# Configuración general
CONFIG = {
    # Datos
    'data_dir': './fruits_vegetables_dataset',
    'batch_size': 32,
    'num_workers': 2,
    'train_split': 0.7,
    'val_split': 0.2,
    'test_split': 0.1,
    
    # Modelo
    'model_name': 'resnet18',  # Cambiar por: vgg16, resnet50, densenet121, etc.
    'pretrained': True,
    'freeze_backbone': True,
    
    # Entrenamiento
    'learning_rate': 0.001,
    'weight_decay': 1e-4,
    'epochs': 50,
    'patience': 10,  # Para early stopping
    'min_delta': 0.001,
    
    # Regularización (Versión 2)
    'dropout_rate': 0.3,
    'use_batch_norm': True,
    
    # Imagen
    'image_size': 224,
    'normalize_mean': [0.485, 0.456, 0.406],  # ImageNet
    'normalize_std': [0.229, 0.224, 0.225],   # ImageNet
}

print("Configuración cargada:")
for key, value in CONFIG.items():
    print(f"  {key}: {value}")

## 📁 3. Descarga y Preparación del Dataset

In [ ]:
# Tu dataset ya está listo en la carpeta test/ ✅
print("📁 DATASET DETECTADO:")
print("   📂 Carpeta: ./test")
print("   🏷️ Clases: 36 (apple, banana, garlic, etc.)")
print("   📊 Total imágenes: ~359")
print("   ✅ Estructura correcta por carpetas")
print()
print("💡 Configuración optimizada para tu dataset:")
print("   🎯 Batch size: 16 (mejor para dataset pequeño)")
print("   ⚡ Épocas: 20 (entrenamiento más rápido)")
print("   📈 Early stopping: 5 épocas de paciencia")

# Verificar que el dataset existe
import os
if os.path.exists('./test'):
    subdirs = [d for d in os.listdir('./test') if os.path.isdir(os.path.join('./test', d))]
    print(f"\n✅ Dataset verificado: {len(subdirs)} clases encontradas")
    print(f"   Primeras clases: {subdirs[:5]}")
else:
    print("❌ Error: No se encuentra la carpeta './test'")

print("\n🚀 ¡Listo para entrenar los modelos!")
print("   Ejecuta las siguientes celdas para continuar...")

## 🔄 4. Transformaciones y Carga de Datos

In [ ]:
# Importar utilidades del proyecto
from data_utils import get_data_transforms, create_data_loaders, visualize_dataset_info, show_sample_images
from models import create_model, print_model_summary
from training_utils import train_model, evaluate_model, compare_models
from config import get_config, print_config, validate_config

# Obtener configuración para el experimento
config = get_config('v1_simple')  # Cambiar por el experimento deseado
print_config(config)
validate_config(config)

# Crear transformaciones de datos
print("\n🔄 Creando transformaciones de datos...")
data_transforms = get_data_transforms(config)

print("✅ Transformaciones creadas:")
print("📈 Entrenamiento: Redimensión + Augmentación + Normalización")
print("📊 Validación/Test: Redimensión + Normalización")

# Cargar y dividir el dataset
print("\n📁 Cargando dataset...")
try:
    dataloaders, datasets, class_names = create_data_loaders(config, data_transforms)
    
    # Mostrar información del dataset
    visualize_dataset_info(dataloaders, class_names)
    
    # Guardar información para uso posterior
    num_classes = len(class_names)
    print(f"\n🏷️ Se detectaron {num_classes} clases:")
    for i, class_name in enumerate(class_names):
        print(f"  {i:2d}. {class_name}")
    
except Exception as e:
    print(f"❌ Error cargando dataset: {e}")
    print("💡 Asegúrate de:")
    print("   1. Haber descargado el dataset de Kaggle")
    print("   2. Extraído en la carpeta correcta")
    print("   3. Actualizado la ruta en config.py")
    
    # Crear dataset de ejemplo para demostración
    print("\n🚧 Creando dataset de ejemplo...")
    from data_utils import create_sample_dataset
    sample_dir, sample_classes = create_sample_dataset()
    
    # Para continuar con el ejemplo, usamos clases simuladas
    class_names = sample_classes
    num_classes = len(class_names)
    print(f"📝 Usando {num_classes} clases de ejemplo: {class_names}")

In [ ]:
# Mostrar muestras del dataset
if 'dataloaders' in locals() and dataloaders is not None:
    print("🖼️ Mostrando muestras del dataset...")
    show_sample_images(dataloaders['train'], class_names, num_samples=8)
else:
    print("⚠️ Dataset no cargado, saltando visualización de muestras")
    print("💡 Una vez que cargues el dataset real, podrás ver las muestras aquí")

## 🧠 5. Implementación de Modelos

### Según la rúbrica, implementaremos:
- **Versión 1**: Clasificador simple (solo FC)
- **Versión 2**: Clasificador embudo con BN y Dropout

In [ ]:
# VERSIÓN 1: Clasificador Simple
print("🏗️ CREANDO MODELO V1 - Clasificador Simple")
print("="*50)

model_v1 = create_model(
    model_name=CONFIG['model_name'],
    num_classes=num_classes,
    version='simple',
    pretrained=CONFIG['pretrained']
)

# Mostrar resumen del modelo V1
print_model_summary(model_v1)

print("\n" + "="*50)

# VERSIÓN 2: Clasificador Embudo (con regularización)
print("\n🏗️ CREANDO MODELO V2 - Clasificador Embudo")
print("="*50)

model_v2_with_reg = create_model(
    model_name=CONFIG['model_name'],
    num_classes=num_classes,
    version='funnel',
    pretrained=CONFIG['pretrained'],
    use_batch_norm=True,
    dropout_rate=CONFIG['dropout_rate']
)

# Mostrar resumen del modelo V2
print_model_summary(model_v2_with_reg)

print("\n" + "="*50)

# VERSIÓN 2 SIN REGULARIZACIÓN (para comparación)
print("\n🏗️ CREANDO MODELO V2 - Sin Regularización")
print("="*50)

model_v2_no_reg = create_model(
    model_name=CONFIG['model_name'],
    num_classes=num_classes,
    version='funnel',
    pretrained=CONFIG['pretrained'],
    use_batch_norm=False,
    dropout_rate=0.0
)

print_model_summary(model_v2_no_reg)

## 🎯 6. Entrenamiento de los Modelos

### Entrenaremos los 3 modelos requeridos por la rúbrica:
1. **V1**: Clasificador simple
2. **V2 sin regularización**: Embudo básico  
3. **V2 con regularización**: Embudo + BN + Dropout

In [ ]:
# ENTRENAMIENTO MODELO V1
print("🚀 ENTRENANDO MODELO V1 - Clasificador Simple")
print("="*60)

# Solo entrenar si tenemos dataloaders válidos
if 'dataloaders' in locals() and dataloaders is not None:
    # Configuración para V1
    config_v1 = get_config('v1_simple')
    
    # Entrenar modelo
    trained_model_v1, history_v1 = train_model(
        model=model_v1,
        dataloaders={'train': dataloaders['train'], 'val': dataloaders['val']},
        config=config_v1,
        model_name="V1 - Clasificador Simple"
    )
    
    # Mostrar curvas de entrenamiento
    history_v1.plot_curves("Curvas de Entrenamiento - Modelo V1")
    
    print("✅ Modelo V1 entrenado exitosamente")
    
else:
    print("⚠️ Dataloaders no disponibles, saltando entrenamiento")
    print("💡 Una vez que cargues el dataset, el entrenamiento se ejecutará aquí")